In [63]:
import os, random
import pandas as pd
import numpy as np
# Pillow = PIL = Python library for image processing
from PIL import Image
from glob import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import torchvision.models as models

print(torch.__version__)
print(torchvision.__version__)
print(np.__version__)

# configs
TRAIN_DIR = "./data/train"
TEST_DIR ="./data/test"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH = 32
NUM_CLASSES = 4

2.9.1
0.24.1
2.3.5


## HELPER FUNCTIONS

In [ ]:
# HELPER FUNCTIONS

# parses labels from train filenames into ID and label
def parse_train_files(train_dir):
  files = glob(os.path.join(train_dir, "*.png"))
  rows=[]
  # HvD — Heavy Damage
  # MiD — Minor Damage
  # MoD — Moderate Damage
  # UD — Undamaged
  label_map = {"HvD":0,"MiD":1,"MoD":2,"UD":3}
  for file in files:
    name = os.path.basename(file)
    id_part, label_part = name.split("_")
    label = label_map[label_part.replace(".png","")]
    rows.append((file,label))
  
  return pd.DataFrame(rows, columns = ["path", "label"])

# batch loader
def build_batch_tensor(batch, transform):
  import io
  images, labels = [], []
  for path, label in batch:
    img = Image.open(path).convert("RGB")
    # resize first
    img = T.Resize((224, 224))(img)
    # random flip if training
    if isinstance(transform.transforms[1], T.RandomHorizontalFlip):
      img = T.RandomHorizontalFlip()(img)
    # convert PIL image to bytes and then to tensor
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PPM')
    img_bytes.seek(0)
    # read PPM format directly and convert to tensor
    img_tensor = torch.frombuffer(img.tobytes(), dtype=torch.uint8).float() / 255.0
    img_tensor = img_tensor.reshape(224, 224, 3).permute(2, 0, 1)
    # apply normalization
    img_tensor = T.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])(img_tensor)
    images.append(img_tensor)
    labels.append(label)
  return torch.stack(images), torch.tensor(labels)


## PREPARING DATA

In [73]:
df = parse_train_files(TRAIN_DIR)

# stratified split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df.label, random_state=42)

# training transformation
train_transform = T.Compose([
  T.Resize((224, 224)),
  T.RandomHorizontalFlip(),
  T.ToTensor(),
  T.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])
])
# validation transformation
val_transform = T.Compose([
  T.Resize((224, 224)),
  T.ToTensor(),
  T.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])
])

# convert df to list of samples --> [(path, label), ...]
train_samples = train_df.values.tolist()
val_samples = val_df.values.tolist()

# load training data
train_loader = DataLoader(
  train_samples,
  batch_size=BATCH,
  shuffle=True,
  num_workers=0,
  collate_fn=lambda b: build_batch_tensor(b, train_transform)
)

# load validation data
val_loader = DataLoader(
  val_samples,
  batch_size=BATCH,
  shuffle=False,
  num_workers=0,
  collate_fn=lambda b: build_batch_tensor(b, val_transform)
)

# create model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

for p in model.parameters():
  p.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

opt = torch.optim.Adam(model.fc.parameters(), lr=1e-3)
lossfn = nn.CrossEntropyLoss()


## TRAINING

In [74]:
# training loop
for epoch in range(3):
  model.train()

  for xb, yb in train_loader:
    xb, yb = xb.to(DEVICE), yb.to(DEVICE)

    preds = model(xb)
    loss = lossfn(preds, yb)

    opt.zero_grad()
    loss.backward()
    opt.step()
  
  # validation
  model.eval()
  correct, total = 0, 0

  with torch.no_grad():
      for xb, yb in val_loader:
          xb, yb = xb.to(DEVICE), yb.to(DEVICE)
          out = model(xb).argmax(dim=1)
          correct += (out == yb).sum().item()
          total += yb.size(0)

  print(f"epoch {epoch} val acc {correct / total:.4f}")

/var/folders/m8/81ptgb3x7j755glt7xp0jptc0000gn/T/ipykernel_23781/3754787080.py:36: UserWarning: The given buffer is not writable, and PyTorch does not support non-writable tensors. This means you can write to the underlying (supposedly non-writable) buffer using the tensor. You may want to copy the buffer to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:1587.)
  img_tensor = torch.frombuffer(img.tobytes(), dtype=torch.uint8).float() / 255.0


epoch 0 val acc 0.6147
epoch 1 val acc 0.6365
epoch 1 val acc 0.6365
epoch 2 val acc 0.6486
epoch 2 val acc 0.6486


## TESTING

In [ ]:
# predicting test set
test_files = sorted(os.listdir(TEST_DIR), key=lambda x: int(x.split(".")[0]))
rows = []

model.eval()
with torch.no_grad():
  for fn in test_files:
    img = Image.open(os.path.join(TEST_DIR, fn)).convert("RGB")
    # Manual preprocessing: resize -> convert to tensor -> normalize (avoid torchvision.ToTensor)
    img = T.Resize((224, 224))(img)
    # Convert raw bytes to tensor (RGB)
    raw = img.tobytes()
    img_tensor = torch.frombuffer(raw, dtype=torch.uint8).float() / 255.0
    img_tensor = img_tensor.reshape(224, 224, 3).permute(2, 0, 1)
    img_tensor = T.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])(img_tensor)
    x = img_tensor.unsqueeze(0).to(DEVICE)
    p = model(x).argmax(dim=1).item()
    rows.append((fn, int(p)))

pd.DataFrame(rows, columns=["ID", "Label"]).to_csv("CL_first_submission.csv", index=False)
print("Wrote CL_first_submission.csv")


Wrote submission_day1.csv
